# Forte Exercise 1.00: Running psi4 jobs in Jupyter notebooks 
## **Goal**: find the symmetry of the triple ground state of linear water

---

In this exercise you are going to use psi4's Python API to compute the restricted open-shell Hartree-Fock wave function of triplet linear water and will use the symmetry information to find out the irrep of the final state.

## Import psi4
The first step necessary to run psi4 in Jupyter is to import psi4

In [ ]:
import psi4

## Specify the molecular geometry
Next, we specify the molecular geometry. We will consider an hydrogen molecule with a bond distance of 1 Å and specify the geometry using the xyz format

In [ ]:
charge = 0
multp = 3
rHH = 1.0 # Ångstrom
geom = f'{charge} {multp}\nO\nH 1 1.0\nH 1 1.0 2 180.0' # we use formatted strings
print(geom)

## Creating the molecule object and accessing its properties
We can now pass the geometry to psi4 and build a `Molecule` object. (see `psi4/src/libmints/molecule.h`)

In [ ]:
mol = psi4.geometry(geom)

The molecule object can queried for information

In [ ]:
print(f"Number of atoms = {mol.natom()}")
print(f"Nuclear repulsion energy = {mol.nuclear_repulsion_energy()}")

and we can even list information about all functions

## Generate Hartree-Fock orbitals using psi4
Using the molecule object we can now compute the MOs using psi4. We first set the options:
1. basis: the basis set (string)
2. scf_type: the type of SCF computation (string). 'pk' stands for the SCF algorithm with conventional integrals 

In [ ]:
# set basis/options
basis = 'cc-pVDZ'
reference = 'rohf'

psi4.core.clean()

psi4.set_options({'basis': basis,'scf_type': 'pk', 'reference' : reference})

# pipe output to the file output.dat
psi4.core.set_output_file('output.dat', False)

# run scf and return the energy and a wavefunction object (will work only if pass return_wfn=True)
E_scf, wfn = psi4.energy('scf', return_wfn=True)

In [ ]:
print(f'SCF Energy: {E_scf}')

# Extracting useful information from psi4
The wavefunction object returned by psi4 is full of useful information. For example, we can ask how many orbitals are there in total

In [ ]:
print(f"Number of orbitals = {wfn.nmo()}")
print(f"Number of alpha electrons = {wfn.nalpha()}")

or we can get information about symmetry

In [ ]:
print(f"Number of irreducible representations (irreps) = {wfn.nirrep()}")
nirrep = wfn.nirrep()

In this case, psi4 detects D2h symmetry, which has eight irreducible representations. We can also find out how many orbitals there are for each irrep.

In [ ]:
# number of occupied molecular orbitals per irrep (mopi). Stored as a Dimension object
nmopi = wfn.nmopi()

# here we convert the psi4 Dimension object to a python tuple
print(f"Number of orbitals in each irreducible representation = {nmopi.to_tuple()}")

## Using the symmetry information in psi4
General symmetry information can be found in the molecule object (because this information is independent of the details of the computation like basis set, method, etc). The information is found in the `PointGroup` object contained in the `Molecule` class.

In [ ]:
point_group = mol.point_group()
print(f'Point group = {point_group.symbol()}')
point_group_symbol = point_group.symbol()

char_table = point_group.char_table()
for h in range(nirrep):
    print(f'Irrep {h} = {char_table.gamma(h).symbol()}')
    
# let's grab the irrep labels frpm psi4's character table
irrep_labels = [char_table.gamma(h).symbol() for h in range(nirrep)]    

In [ ]:
print(f"Number of alpha electrons in each irreducible representation = {wfn.nalphapi().to_tuple()}")
print(f"Number of beta electrons in each irreducible representation = {wfn.nbetapi().to_tuple()}")

In [ ]:
sym = 0

# compute the product of the irrep of all the occupied orbitals
for h, nh in enumerate(wfn.nalphapi().to_tuple()):
    if nh % 2 == 1: sym ^= h 
for h, nh in enumerate(wfn.nbetapi().to_tuple()):
    if nh % 2 == 1: sym ^= h
        
print(f'Symmetry = {irrep_labels[sym]}')